In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os


GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Computer Vision/Coursework/' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['CW_Dataset-2.zip', 'Untitled0.ipynb', 'EmotionRecognition.py', '__pycache__', 'ASARJOU_CV_VIDEO.avi', 'ASARJOU_CV_VIDEO-3.avi', 'ASARJOU_CV_CW_SHORT.mp4', 'ASARJOU_CV_CW_SHORT.avi', 'train_lab.csv', 'val_lab.csv', 'fin_MLP_ADASYN.pkl', 'fin_CNN.pkl', 'fin_SIFT_MLP.pkl']


In [ ]:
# FROM LAB 9 #
# Identify path to zipped dataset
zip_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset-2.zip')

# Copy it to Colab
!cp '{zip_path}' .

# Unzip it (removing useless files stored in the zip)
!yes|unzip -q CW_Dataset-2.zip

# Delete zipped version from Colab (not from Drive)
!rm CW_Dataset-2.zip

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install opencv-python==4.4.0.46

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, Lambda
import time
import pandas as pd
import copy
from PIL import Image
from skimage import io, img_as_ubyte



%matplotlib inline

In [ ]:
class CustomImageDataset(Dataset):

    def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):

        self.img_labels = annotations_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = io.imread(img_path)
        h = image.copy()

        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

      
        sample = {"image": image, "label": label}
        return image, label

In [ ]:
class MLP(nn.Module):
    def __init__(
            self,
            hidden_num = 40,
            dropout = 0.1,
            nonlin = torch.nn.Sigmoid()

    
    ):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(70,hidden_num)
        self.fc2 = nn.Linear(hidden_num,8)
        self.non_lin = nonlin
        self.dropout = dropout
        
    def forward(self, x):
        hidden = self.fc1(x)
        hidden = self.non_lin(hidden)
        output = self.fc2(hidden)
        return output

In [ ]:
data_dir = "CW_Dataset-2"
l_test = os.path.join(data_dir, "labels", "list_label_test.txt")
d_test = os.path.join(data_dir, "test")
d = os.path.join(data_dir, "train")

In [ ]:
train_lab = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'train_lab.csv'))
train_lab = train_lab.drop(columns='Unnamed: 0')
val_lab = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'val_lab.csv'))
val_lab = val_lab.drop(columns='Unnamed: 0')
test_lab = pd.read_csv(l_test, delimiter= ' ',header=None)
test_lab = test_lab.rename(columns={0: "name", 1: "lbl"})
test_lab['name'] = test_lab['name'].str[:-4] + "_aligned" + ".jpg"
test_lab['lbl'] = test_lab['lbl'] - 1

In [ ]:
data_means = [0.485, 0.456, 0.406]
data_stds = [0.229, 0.224, 0.225]

training_data = CustomImageDataset(train_lab, d, transform=transforms.Compose([transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomGrayscale(),
        transforms.RandomRotation(10),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)]))

validation_data = CustomImageDataset(val_lab, d, transform=transforms.Compose([transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)]))

test_data = CustomImageDataset(test_lab, d_test, transform=transforms.Compose([transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)]))

In [ ]:
ds_train = DataLoader(training_data, batch_size =4, shuffle=True)
ds_val = DataLoader(validation_data, batch_size =4, shuffle=True)
ds_test = DataLoader(test_data, batch_size =4, shuffle=False)

In [ ]:
### ADAPTED FROM LAB 8 (LAB IMPLEMENTATION OF SIFT BOVW) ###

def sift_conv(ds, b_num):
  des_list = []
  l_set = []
  for image, label in ds: #taking images and applying sift 
    for i in range(b_num-1):
      #print(image[i].shape)
      inp = image[i].numpy().transpose((1, 2, 0))
      image8bit = cv2.normalize(inp, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
      sift = cv2.SIFT_create()
      SIFT_kp, SIFT_des = sift.detectAndCompute(image8bit, None)
      if SIFT_des is not None:
          des_list.append(SIFT_des)
          l_set.append(label[i])
  des_array = np.vstack(des_list)
  return des_array, des_list, l_set

In [ ]:
train_arr, train_list, train_labels = sift_conv(ds_train, 4)
val_arr, val_list, val_labels = sift_conv(ds_val, 4)
test_arr, test_list, test_labels = sift_conv(ds_test, 4)

In [ ]:
### ADAPTED FROM LAB 8 (LAB IMPLEMENTATION OF SIFT BOVW) ###

from sklearn.cluster import MiniBatchKMeans
def kmeans_bovw(des_array, des_list):
  batch_size = des_array.shape[0] // 4
  kmeans = MiniBatchKMeans(n_clusters=70, batch_size=batch_size).fit(des_array) #Doing KMeans for BoVW

  # Convert descriptors into histograms of codewords for each image
  hist_list = []
  idx_list = []

  for des in des_list:
      hist = np.zeros(70)

      idx = kmeans.predict(des)
      idx_list.append(idx)
      for j in idx:
          hist[j] = hist[j] + (1 / len(des))
      hist_list.append(hist)

  hist_array = np.vstack(hist_list)
  return hist_array

In [ ]:
train_bovw = kmeans_bovw(train_arr, train_list)
val_bovw = kmeans_bovw(val_arr, val_list)
test_bovw = kmeans_bovw(test_arr, test_list)

In [ ]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(torch.tensor(train_bovw), torch.tensor(train_labels))
val_ds = TensorDataset(torch.tensor(val_bovw), torch.tensor(val_labels))

train_dl = DataLoader(train_ds, batch_size=4)
val_dl = DataLoader(val_ds, batch_size = 4)

In [ ]:
training_dataset_sizes = len(train_labels)
validation_dataset_sizes = len(val_labels)

In [ ]:
### ADDAPTED FROM LAB 9 ###
def train_model(model, criterion, optimizer, num_epochs=10):
    since = time.time()

    best_acc = 0.0

    for epoch in range(num_epochs):
        
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print('----------------')

        # Each epoch has a training and validation phase
        
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

            # Iterate over data
        for inputs, labels in train_dl:
       
                # move data to GPU
                input = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(True):
           
                    outputs = model(input.float())
                    _, preds = torch.max(outputs, 1)
      
                    
                    loss = criterion(outputs, labels)
             
                    # backward + optimize only if in training phase

                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item() * input.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # update learning rate with scheduler


        epoch_loss = running_loss / training_dataset_sizes
        epoch_acc = running_corrects.double() / training_dataset_sizes

        print(f"train loss: {epoch_loss:.4f} train acc: {epoch_acc:.4f}")
      
        
        model.eval()  # Set model to evaluation mode
        running_loss = 0.0
        running_corrects = 0

        for inputs_val, labels_val, in val_dl:
                # move data to GPU
                input_val = inputs_val.to(device)
                labels_val = labels_val.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    outputs_val = model(input_val.float())
               
                    _, preds_val = torch.max(outputs_val, 1)
                    loss_val = criterion(outputs_val, labels_val)

                # statistics
                running_loss += loss_val.item() * input_val.size(0)
                running_corrects += torch.sum(preds_val == labels_val.data)


        epoch_loss = running_loss / validation_dataset_sizes
        epoch_acc = running_corrects.double() / validation_dataset_sizes

        print(f"val loss: {epoch_loss:.4f} val acc: {epoch_acc:.4f}")



    time_elapsed = time.time() - since

    print(time_elapsed)
    torch.save(model,os.path.join(GOOGLE_DRIVE_PATH,'fin_SIFT_MLP.pth'))
    return model, best_acc

In [ ]:
model = MLP()

model = model.to(device)
    
    # Define criterion
criterion = nn.CrossEntropyLoss()
    
    # optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0008697902379323465)
    
    # Train model
best_model= train_model(model, criterion, optimizer, num_epochs=20)

Epoch 0/19
----------------
train loss: 1.6593 train acc: 0.3781
val loss: 1.6770 val acc: 0.3829
Epoch 1/19
----------------
train loss: 1.6386 train acc: 0.3861
val loss: 1.6768 val acc: 0.3829
Epoch 2/19
----------------
train loss: 1.6338 train acc: 0.3861
val loss: 1.6772 val acc: 0.3829
Epoch 3/19
----------------
train loss: 1.6278 train acc: 0.3860
val loss: 1.6780 val acc: 0.3829
Epoch 4/19
----------------
train loss: 1.6205 train acc: 0.3867
val loss: 1.6795 val acc: 0.3829
Epoch 5/19
----------------
train loss: 1.6122 train acc: 0.3891
val loss: 1.6818 val acc: 0.3835
Epoch 6/19
----------------
train loss: 1.6036 train acc: 0.3913
val loss: 1.6850 val acc: 0.3851
Epoch 7/19
----------------
train loss: 1.5952 train acc: 0.3984
val loss: 1.6890 val acc: 0.3819
Epoch 8/19
----------------
train loss: 1.5877 train acc: 0.4030
val loss: 1.6937 val acc: 0.3813
Epoch 9/19
----------------
train loss: 1.5809 train acc: 0.4064
val loss: 1.6988 val acc: 0.3802
Epoch 10/19
--------

In [ ]:
model